<a href="https://colab.research.google.com/github/hernandemonteiro/market-analysis/blob/main/analise_de_vendas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Análise de vendas**

Estamos recebendo algumas vendas em nosso ecommerce e recebemos a tarefa de responder algumas perguntas provenientes da administração de negócios.

## **Perguntas:**
- Houve algum desconto nas vendas?
- Qual tipo de pagamento devemos prestar mais atenção?
- Existe diferença de entre método de pagamento em regiões diferentes?
- Existe algum produto que se diferencia pelo método de pagamento do que os demais?


In [1]:
import pandas as pd

**Recebemos os dados**

In [6]:
data = pd.read_csv("/content/drive/MyDrive/Data Science/dados/Online Sales Data.csv")
data.head()

,Transaction ID,Date,Product Category,Product Name,Units Sold,Unit Price,Total Revenue,Region,Payment Method
0,10001,2024-01-01,Electronics,iPhone 14 Pro,2,999.99,1999.98,North America,Credit Card
1,10002,2024-01-02,Home Appliances,Dyson V11 Vacuum,1,499.99,499.99,Europe,PayPal
2,10003,2024-01-03,Clothing,Levi's 501 Jeans,3,69.99,209.97,Asia,Debit Card
3,10004,2024-01-04,Books,The Da Vinci Code,4,15.99,63.96,North America,Credit Card
4,10005,2024-01-05,Beauty Products,Neutrogena Skincare Set,1,89.99,89.99,Europe,PayPal


## Houve algum desconto nas vendas?

In [20]:
data["discount"] = round(data["Units Sold"] * data["Unit Price"], 2) - data["Total Revenue"]
data.head()

,Transaction ID,Date,Product Category,Product Name,Units Sold,Unit Price,Total Revenue,Region,Payment Method,discount
0,10001,2024-01-01,Electronics,iPhone 14 Pro,2,999.99,1999.98,North America,Credit Card,0.0
1,10002,2024-01-02,Home Appliances,Dyson V11 Vacuum,1,499.99,499.99,Europe,PayPal,0.0
2,10003,2024-01-03,Clothing,Levi's 501 Jeans,3,69.99,209.97,Asia,Debit Card,0.0
3,10004,2024-01-04,Books,The Da Vinci Code,4,15.99,63.96,North America,Credit Card,0.0
4,10005,2024-01-05,Beauty Products,Neutrogena Skincare Set,1,89.99,89.99,Europe,PayPal,0.0


In [25]:
is_discount = len(data[data["discount"] > 0])

if is_discount:
  print("Houve desconto!")
else:
  print("Não houve desconto!")

# nesse caso sabemos que não houve desconto

Não houve desconto!


## Qual tipo de pagamento devemos prestar mais atenção?

In [32]:
payment_methods = data["Payment Method"]
payment_method_counts = payment_methods.value_counts()
print("contagem de aparições")
payment_method_counts

contagem de aparições


Payment Method
Credit Card    120
PayPal          80
Debit Card      40
Name: count, dtype: int64

In [38]:
dict_translate = {
    "Credit Card": "Cartão de Crédito",
    "Debit Card": "Debit Card",
    "Paypal": "Paypal"
}

In [41]:
print("O mais utilizado e portanto que se deve mais atenção é:",
      dict_translate[payment_methods.mode()[0]])

O mais utilizado e portanto que se deve mais atenção é: Cartão de Crédito


## Existe diferença de entre método de pagamento em regiões diferentes?

In [84]:
grouped = data.groupby('Region')['Payment Method'].unique()

# Convertendo para um formato mais legível
payments_per_region = {region: methods.tolist() for region, methods in grouped.items()}

# Contando métodos de pagamento únicos por região
unique_payments_per_region = {region: len(methods) for region, methods in payments_per_region.items()}

# Identificando diferenças entre regiões
print("Métodos de pagamento por região:")
for region, methods in payments_per_region.items():
    print(f"- {region}: {', '.join(methods)}")

print("\nNúmero de métodos de pagamento únicos por região:")
for region, count in unique_payments_per_region.items():
    print(f"- {region}: {count}")

# Verificando se há métodos exclusivos por região
exclusive_methods = {}
for region, methods in payments_per_region.items():
    exclusive_methods[region] = set(methods)
    for other_region, other_methods in payments_per_region.items():
        if region != other_region:
            exclusive_methods[region] -= set(other_methods)

print("\nMétodos de pagamento exclusivos por região:")
for region, methods in exclusive_methods.items():
    if methods:
        print(f"- {region}: {', '.join(methods)}")
    else:
        print(f"- {region}: Nenhum método exclusivo")

# Comparando métodos entre regiões
print("\nComparação de métodos de pagamento entre regiões:")
for region1, methods1 in payments_per_region.items():
    for region2, methods2 in payments_per_region.items():
        if region1 < region2:  # Evitando as comparações duplicadas
            common_methods = set(methods1) & set(methods2)
            if common_methods:
                print(f"- {region1} e {region2} compartilham os métodos: {', '.join(common_methods)}")
            else:
                print(f"- {region1} e {region2} não têm métodos de pagamento em comum")


Métodos de pagamento por região:
- Asia: Debit Card, Credit Card
- Europe: PayPal
- North America: Credit Card

Número de métodos de pagamento únicos por região:
- Asia: 2
- Europe: 1
- North America: 1

Métodos de pagamento exclusivos por região:
- Asia: Debit Card
- Europe: PayPal
- North America: Nenhum método exclusivo

Comparação de métodos de pagamento entre regiões:
- Asia e Europe não têm métodos de pagamento em comum
- Asia e North America compartilham os métodos: Credit Card
- Europe e North America não têm métodos de pagamento em comum


## Existe algum produto que se diferencia pelo método de pagamento do que os demais?

In [80]:
payment_counts = data.groupby("Product Name")["Payment Method"].nunique()
payment_counts

Product Name
1984 by George Orwell                                 1
Adidas 3-Stripes Shorts                               1
Adidas Essential Track Pants                          1
Adidas FIFA World Cup Football                        1
Adidas Originals Superstar Sneakers                   1
                                                     ..
Yeti Tundra Haul Portable Wheeled Cooler              1
Youth to the People Superfood Antioxidant Cleanser    1
Zara Summer Dress                                     1
iPhone 14 Pro                                         1
iRobot Braava Jet M6                                  1
Name: Payment Method, Length: 232, dtype: int64

In [81]:
products_with_different_payments = payment_counts[payment_counts > 1]

if len(products_with_different_payments) > 0:
    print("Sim, há produto(s) que se diferenciam pelo método de pagamento:")
    for product, count in products_with_different_payments.items():
        print(f"- {product}: {count} métodos diferentes")
else:
    print("Não, todos os produtos têm o mesmo método de pagamento associado.")

Não, todos os produtos têm o mesmo método de pagamento associado.
